# Osmosis Exploit Analysis

Breakdown of exploiting done by users and to what extent, i.e. extra assets taken from pools.

Goal is to Calculate:

- Count of Exit & Join Transactions (In Total, By Wallet)

- Number of Unique Wallets w/ Join/Exit transactions.

- Wallets w/ Joins but no exits

- Wallets w/ Joins & Exits

## Imports and Preliminaries

In [1]:
import requests
import json

import numpy as np
import pandas as pd
import pymysql

In [2]:
halt_prices = [{"symbol": "ION", "denom": "uion", "price": 901.2415907289011}
,{"symbol": "ATOM", "denom": "ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEADA9CA97EA622B25F41E5EB2", "price": 9.419861882554208}
,{"symbol": "AKT", "denom": "ibc/1480B8FD20AD5FCAE81EA87584D269547DD4D436843C1D20F15E00EB64743EF4", "price": 0.34922783600524077}
,{"symbol": "XPRT", "denom": "ibc/A0CC0CF735BFB30E730C70019D4218A1244FF383503FF7579C9201AB93CA9293", "price": 1.381620130778327}
,{"symbol": "IRIS", "denom": "ibc/7C4D60AA95E5A7558B0A364860979CA34B7FF8AAF255B87AF9E879374470CEC0", "price": 0.0244467298543281}
,{"symbol": "DVPN", "denom": "ibc/9712DBB13B9631EDFA9BF61B55F1B2D290B2ADB67E3A4EB3A875F3B6081B3B84", "price": 0.0014306305334162999}
,{"symbol": "REGEN", "denom": "ibc/1DCC8A6CB5689018431323953344A9F6CC4D0BFB261E88C9F7777372C10CD076", "price": 0.32060717495837876}
,{"symbol": "CRO", "denom": "ibc/E6931F78057F7CC5DA0FD6CEF82FF39373A6E0452BF1FD76910B93292CF356C1", "price": 0.17790777510495}
,{"symbol": "IOV", "denom": "ibc/52B1AA623B34EB78FD767CEA69E8D7FA6C9CFE1FBF49C5406268FD325E2CC2AC", "price": 0.0276612683659155}
,{"symbol": "TICK", "denom": "ibc/655BCEF3CDEBE32863FF281DBBE3B06160339E9897DC9C9C9821932A5F8BA6F8", "price": 0.4027368348934557}
,{"symbol": "EEUR", "denom": "ibc/5973C068568365FFF40DEDCF1A1CB7582B6116B731CD31A12231AE25E20B871F", "price": 1.0136578340008713}
,{"symbol": "NGM", "denom": "ibc/1DC495FCEFDA068A3820F903EDBD78B942FBD204D7E93D3BA2B432E9669D1A59", "price": 0.6919927498542117}
,{"symbol": "BCNA", "denom": "ibc/D805F1DA50D31B96E4282C1D4181EDDFB1A44A598BFF5666F4B43E4B8BEA95A5", "price": 0.014288225525700901}
,{"symbol": "LUNA", "denom": "ibc/0EF15DF2F02480ADE0BB6E85D9EBB5DAEA2836D3860E9F97F9AADE4F57A31AA0", "price": 4.86041604111e-05}
,{"symbol": "LIKE", "denom": "ibc/9989AD6CCA39D1131523DB0617B50F6442081162294B4795E26746292467B525", "price": 0.0024598378860000002}
,{"symbol": "UST", "denom": "ibc/BE1BB42D4BE3C30D50B68D7C41DB4DFCE9678E8EF8C539F6E6A9345048894FCC", "price": 0.0086640074588253}
,{"symbol": "JUNO", "denom": "ibc/46B44899322F3CD854D2D46DEEF881958467CDD4B3B10086DA49296BBED94BED", "price": 3.7344029425363066}
,{"symbol": "IXO", "denom": "ibc/F3FF7A84A73B62921538642F9797C423D2B4C4ACB3C7FCFFCE7F12AA69909C4B", "price": 0.1074952398695577}
,{"symbol": "XKI", "denom": "ibc/B547DC9B897E7C3AA5B824696110B8E3D2C31E3ED3F02FF363DCBAD82457E07E", "price": 0.0786558545103051}
,{"symbol": "BTSG", "denom": "ibc/4E5444C35610CC76FC94E7F7886B93121175C28262DDFDDE6F84E82BF2425452", "price": 0.0166805855860554}
,{"symbol": "BOOT", "denom": "ibc/FE2CD1E6828EC0FAB8AF39BAC45BC25B965BA67CCBC50C13A14BD610B0D1E2C4", "price": 2.2282665111957e-08}
,{"symbol": "SCRT", "denom": "ibc/0954E1C28EB7AF5B72D24F3BC2B47BBB2FDF91BDDFD57B74B99E133AED40972A", "price": 1.303759474770078}
,{"symbol": "MED", "denom": "ibc/3BCCC93AD5DF58D11A6F8A05FA8BC801CBA0BA61A981F57E91B8B598BF8061CB", "price": 0.028320572005851297}
,{"symbol": "CMDX", "denom": "ibc/EA3E1640F9B1532AB129A571203A0B9F789A7F14BB66E350DCBFA18E1A1931F0", "price": 0.3443737484561466}
,{"symbol": "CHEQ", "denom": "ibc/7A08C6F11EF0F59EB841B9F788A87EC9F2361C7D9703157EC13D940DC53031FA", "price": 0.039513941314199996}
,{"symbol": "STARS", "denom": "ibc/987C17B11ABC2B20019178ACE62929FE9840202CE79498E29FE8E5CB02B7C0A4", "price": 0.032098245677113196}
,{"symbol": "LUM", "denom": "ibc/8A34AF0C1943FD0DFCDE9ADBF0B2C9959C45E87E6088EA2FC6ADACD59261B8A2", "price": 0.0013626160158684}
,{"symbol": "VDL", "denom": "ibc/E7B35499CFBEB0FF5778127ABA4FB2C4B79A6B8D3D831D4379C4048C238796BD", "price": 0.0388842787207905}
,{"symbol": "HUAHUA", "denom": "ibc/B9E0A1A524E98BB407D3CED8720EFEFD186002F90C1B1B7964811DD0CCC12228", "price": 4.69158171348e-05}
,{"symbol": "DSM", "denom": "ibc/EA4C0A9F72E2CEDF10D0E7A9A6A22954DB3444910DB5BE980DF59B05A46DAD1C", "price": 0.020904272590374296}
,{"symbol": "DIG", "denom": "ibc/307E5C96C8F60D1CBEE269A9A86C0834E1DB06F2B3788AE4F716EDB97A48B97D", "price": 0.0003890904481587}
,{"symbol": "SOMM", "denom": "ibc/9BBA9A1C257E971E38C1422780CE6F0B0686F0A3085E2D61118D904BFE0F5F5E", "price": 0.481296017774841}
,{"symbol": "DARC", "denom": "ibc/346786EA82F41FE55FAD14BF69AD8BA9B36985406E43F3CB23E6C45A285A9593", "price": 0.0027962542601544}
,{"symbol": "BAND", "denom": "ibc/F867AE2112EFE646EC71A25CD2DFABB8927126AC1E19F1BBF0FF693A4ECA05DE", "price": 1.6658623706165938}
,{"symbol": "NETA", "denom": "ibc/297C64CC42B5A8D8F82FE2EBE208A6FE8F94B86037FA28C4529A23701C228F7A", "price": 62.24130405901527}
,{"symbol": "UMEE", "denom": "ibc/67795E528DF67C5606FC20F824EA39A6EF55BA133F4DC79C90A8C47A0901E17C", "price": 0.0303993250979034}
,{"symbol": "GRAV", "denom": "ibc/E97634A40119F1898989C2A23224ED83FDD0A57EA46B3A094E287288D1672B44", "price": 0.0121737712410579}
,{"symbol": "DEC", "denom": "ibc/9BCB27203424535B6230D594553F1659C77EC173E36D9CF4759E7186EE747E84", "price": 0.0148596346855374}
,{"symbol": "SWTH", "denom": "ibc/8FEFAE6AECF6E2A255585617F781F35A8D5709A545A804482A261C0C9548A9D3", "price": 0.00513882496548}
,{"symbol": "MARBLE", "denom": "ibc/F6B691D5F7126579DDC87357B09D653B47FDCE0A3383FF33C8D8B544FE29A8A6", "price": 5.896002397261752}
,{"symbol": "ROWAN", "denom": "ibc/8318FD63C42203D16DDCAF49FE10E8590669B3219A3E87676AC9DA50722687FB", "price": 0.01133647787403854}
,{"symbol": "CRBRUS", "denom": "ibc/41999DF04D9441DAC0DF5D8291DF4333FBCBA810FFD63FDCE34FDF41EF37B6F7", "price": 9.2914785603e-06}
,{"symbol": "RAC", "denom": "ibc/6BDB4C8CCD45033F9604E4B93ED395008A753E01EECD6992E7D1EA23D9D3B788", "price": 0.0747209077494774}
,{"symbol": "HOPE", "denom": "ibc/C2A2E9CA95DDD4828B75124B5E27B8401C7D8493BC48353D418CBFC04565899B", "price": 6.920993637006235}
,{"symbol": "axlUSDC", "denom": "ibc/D189335C6E4A68B513C10AB227BF1C1D38C746766278BA3EEB4FB14124F1D858", "price": 1.0000000090199457}
,{"symbol": "axlUSDT", "denom": "ibc/8242AD24008032E457D2E12D46588FD39FB54FB29680C6C7663D296B383C37C4", "price": 0.94426468}
,{"symbol": "PSTAKE", "denom": "ibc/8061A06D3BD4D52C4A28FFECF7150D370393AF0BA661C3776C54FF32836C3961", "price": 0.2072535267442587}
,{"symbol": "axlWBTC", "denom": "ibc/D1542AA8762DB13087D8364F3EA6509FD6F009A34F00426AF9E4F9FA85CBBF1F", "price": 31016.743795684255}
,{"symbol": "gUSDC", "denom": "ibc/9F9B07EF9AD291167CF5700628145DE1DEB777C2CFC7907553B24446515F6D0E", "price": 1.0078654524712436}
,{"symbol": "RAW", "denom": "ibc/00B6E60AD3D65CBEF5579AC8AF609527C0B57535B6E32D96C80A735344FD9DCC", "price": 0.0276260703219831}
,{"symbol": "MNTL", "denom": "ibc/CBA34207E969623D95D057D9B11B0C8B32B89A71F170577D982FDDE623813FFC", "price": 0.09402846602480519}
,{"symbol": "FET", "denom": "ibc/5D1F516200EE8C6B2354102143B78A2DEDA25EDE771AC0F8DC3C1837C8FD4447", "price": 0.15429126801601742}
,{"symbol": "axlDAI", "denom": "ibc/0CD3A0285E1341859B5E86B6AB7682F023D03E97607CCC1DC95706411D866DF7", "price": 0.9950205268899415}
,{"symbol": "HASH", "denom": "ibc/CE5BFF1D9BADA03BB5CCA5F56939392A761B53A10FBD03B37506669C3218D3B2", "price": 0.031977892518}
,{"symbol": "axlFRAX", "denom": "ibc/0E43EDE2E2A3AFA36D0CD38BDDC0B49FECA64FA426A82E102F304E430ECF46EE", "price": 0.8199521681810243}
,{"symbol": "axlWETH", "denom": "ibc/EA1D43981D5C9A1C4AAEA9C23BB1D4FA126BA9BC7020A25E0AE4AA841EA25DC5", "price": 1822.4890264165895}
,{"symbol": "gDAI", "denom": "ibc/F292A17CF920E3462C816CBE6B042E779F676CAB59096904C4C1C966413E3DF5", "price": 0.9918005785267493}
,{"symbol": "gWETH", "denom": "ibc/65381C5F3FD21442283D56925E62EA524DED8B6927F0FF94E21E0020954C40B5", "price": 1775.7727319807198}
,{"symbol": "EVMOS", "denom": "ibc/6AE98883D4D5D5FF9E50D7130F1305DA2FFA0C652D1DD9C123657C6B4EB2DF8A", "price": 1.819049281393931}
,{"symbol": "OSMO", "denom": "uosmo", "price": 1.11810813}]
halt_price_df = pd.DataFrame(halt_prices)

In [3]:
with open('data/tokens_map.json', 'r') as token_file:
    token_map = json.load(token_file)
    token_map_df = pd.DataFrame(token_map)

## Transaction Processing

In [4]:
# Source: https://gist.github.com/joeabbey/74e4d5caf92e31054b7fc0639b868ef1
raw_transactions = pd.read_csv("data/joins_exits.csv", skipinitialspace=True)
pool_transactions = raw_transactions.loc[(raw_transactions.height >=4707300) & (raw_transactions.code == 0)]
pool_transactions.columns

Index(['height', 'code', 'sender', 'type', 'pool_id', 'shares',
       'token_1_amount', 'token_1_denom', 'token_2_amount', 'token_2_denom'],
      dtype='object')

In [5]:
valid_txs = []
for idx, grouped_txs in pool_transactions.sort_values(by=['height']).groupby(['sender', 'pool_id']):
    sender, pool_id = idx
    shares_joined = 0.0
    for _, tx_info in grouped_txs.iterrows():
        # ToDo: Take 64 transactions where below condition is true and normalize them
        # float(tx_info.shares) >= shares_joined:
        if shares_joined and tx_info.type == 'Exit':
            shares_joined -= float(tx_info.shares)
            valid_txs.append(tx_info.values)
        elif tx_info.type == 'Join':
            shares_joined += float(tx_info.shares)
            valid_txs.append(tx_info.values)
valid_transactions = pd.DataFrame(valid_txs, columns=pool_transactions.columns.values)

## Count of Join/Exit Transactions

In [6]:
valid_transactions.groupby('type').size().reset_index(name='number_of_txs')

,type,number_of_txs
0,Exit,277
1,Join,1972


In [7]:
(valid_transactions.groupby(['type', 'sender'])
                   .size()
                   .reset_index(name='number_of_txs')
                   .sort_values(by=['number_of_txs'], ascending=False))

,type,sender,number_of_txs
323,Join,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,77
4,Exit,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,77
1170,Join,osmo1yglld3aary7lnrrn2dz7la84kmnmen4kpsxzay,43
20,Exit,osmo1yglld3aary7lnrrn2dz7la84kmnmen4kpsxzay,42
376,Join,osmo1adtxg4klr2z65zqnxwxv66k5wel7mljutaq6mt,29
...,...,...,...
474,Join,osmo1dlwzczuey4wfue5xqk7mt4ygrzk9krkszhnlgl,1
477,Join,osmo1dmepl046vkmqsxly2qcm9sfnmnhlz6w9lvfxus,1
478,Join,osmo1dncfdqzjw0j2vnak3rzhsru9z7s0f2wclty83c,1
479,Join,osmo1dpsue447gd8jgmxaajjvy4uqwxyvlm4yufzvje,1


## Number of Unique Addresses

- Just Join Txs
- Both Join and Exit Txs

In [8]:
exit_tx_addresses = valid_transactions.loc[valid_transactions.type == 'Exit'].sender.unique()
join_only_users = valid_transactions.loc[
    (valid_transactions.type == 'Join') &
    (~np.isin(valid_transactions.sender, exit_tx_addresses))
].sender.unique()

join_only_users.shape[0]

1168

In [9]:
potentially_malicious_users = valid_transactions.loc[~(np.isin(valid_transactions.sender, join_only_users))].sender.unique()
potentially_malicious_users.shape[0]

22

## Wallet Lists

- Just Join Txs

- Both Join and Exit Txs

In [10]:
join_only_users

array(['osmo103v3wy2qccy47jxvnsnxvj3r2lhxcfavn50n9a',
       'osmo103w2uwpdf9p708c83nkyd0ys6fgqt94tdvjtgn',
       'osmo104gevychrp9gvnyfvewpaaql7f4mp694zkcecm', ...,
       'osmo1zulgguv8hfwvwd5kqd6res3a4zuv3jsgz2yhn7',
       'osmo1zzff42ula9c53v798hj6eu0e9s8y8rdgtuftcw',
       'osmo1zzgc9jukqxppns6lu042a87kej5af6f29shel2'], dtype=object)

In [11]:
potentially_malicious_users

array(['osmo10rgny3xd5fvg02wnmy4h9zuktnfthvrlwdjwf5',
       'osmo10t26acjmemggsahq6uvyucm4tj3z0mhz23ljh2',
       'osmo127cz3qd25qefq2d3l8zdl9xqzatskjcq4n5eg2',
       'osmo16h0yyryfjmmhgj3ct6ndl7gcy4r3krf0gfrzky',
       'osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq',
       'osmo1c5txl47ea73x9g535lefzs8u6engjrss4tr60p',
       'osmo1csl24xa338980yckumj6k9kejyttshaaktn9tv',
       'osmo1cw73elmtjvzg4gjdy9yv7hu6nnfl8m7z4ah5wl',
       'osmo1d8etkcl43lp9nhdsflmm9y3krl7dutwj2493rh',
       'osmo1ea6yv55jzerkeqqz9c36t6wyklh8qpqs0da0n6',
       'osmo1hq8tlgq0kqz9e56532zghdhz7g8gtjymdltqer',
       'osmo1jdh7eeyaar0tyask0r6w2228uh5wrd0pxtcfwr',
       'osmo1jfa3vw3r50vyww2qwklxqn6qu06d3uf7vc63qm',
       'osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte',
       'osmo1pfpk6rsgc0x7g8q7y0yre86ej2enr8rhv3p8dq',
       'osmo1tg70tuzekpd376dpqr68yx5a7r709w6x8jtxha',
       'osmo1ugg4nh0vgmqwvkvkjaw745hh70j5aae6y7m6ux',
       'osmo1ux20lcw7et2j8kl8gfdep78lacew4dqqjp5dvp',
       'osmo1v44mqmhvtn8cw37

## Assets in Pools Impacted by Exploit

Note these assets may or may not be bonded. That hasn't been verified yet. Leaving that part to Geo from Dexmos.

In [12]:
join_txs = valid_transactions.loc[np.isin(valid_transactions.sender, join_only_users)]
join_first_token = join_txs.drop(columns=['token_2_denom', 'token_2_amount']).rename(columns={'token_1_denom':'denom', 'token_1_amount': 'amount'})
join_second_token = join_txs.drop(columns=['token_1_denom', 'token_1_amount']).rename(columns={'token_2_denom':'denom', 'token_2_amount': 'amount'})

flat_join_txs = pd.concat([join_first_token, join_second_token])
flat_join_txs.amount = flat_join_txs.amount.astype(float)
flat_join_txs.shares = flat_join_txs.shares.astype(float, copy=False)

In [13]:
named_join_txs = token_map_df.merge(flat_join_txs)
named_join_txs['amount'] = named_join_txs.amount/(10**named_join_txs.exponent)
named_join_txs = named_join_txs.merge(halt_price_df)
named_join_txs['value'] = named_join_txs.price * named_join_txs.amount
cleaned_join_df = named_join_txs.drop(columns=['denom', 'exponent'])

In [14]:
agg_join_value = cleaned_join_df['value'].sum()
agg_join_value

180765.34513924943

In [27]:
agg_join_sender_tokens = (cleaned_join_df.groupby(['sender', 'symbol'])[['amount', 'value']]
                                         .sum()
                                         .reset_index()
                                         .sort_values(by=['value'], ascending=False))

agg_join_sender_tokens.to_csv("data/agg_join_sender_tokens.csv", index=False)
agg_join_sender_tokens

,sender,symbol,amount,value
1205,osmo1ew02nl40jqspvfe23xmdc4xmqvnelz3929nrrx,OSMO,14092.243250,15756.651748
1204,osmo1ew02nl40jqspvfe23xmdc4xmqvnelz3929nrrx,MNTL,164617.631861,15478.743405
1778,osmo1lvxcql4c9yeec8spuaspeq5j5slnd27g6zdh0a,axlWETH,6.309076,11498.221360
1777,osmo1lvxcql4c9yeec8spuaspeq5j5slnd27g6zdh0a,OSMO,9982.212293,11161.192720
1564,osmo1jh0urskmdrrhfgm53d0ddnd506stu63w95d3l6,ION,10.299381,9282.230516
...,...,...,...,...
1095,osmo1dvslrm9zzkdn6cgn4dyve0vfgent2v5ujn3sd9,DSM,0.002331,0.000049
2448,osmo1vhdwjygytsd5l8d2fk46awka6usltxe3l0hmcm,DIG,0.059118,0.000023
2752,osmo1zl9t43fvpxjatedsqvusp4nedh6qqvcqj0823c,JUNO,0.000001,0.000004
2753,osmo1zl9t43fvpxjatedsqvusp4nedh6qqvcqj0823c,UST,0.000411,0.000004


In [16]:
agg_join_senders = (cleaned_join_df.groupby(['sender'])[['value']]
                                   .sum()
                                   .reset_index()
                                   .sort_values(by=['value'], ascending=False))
agg_join_senders.to_csv("data/agg_join_senders.csv", index=False)
agg_join_senders

,sender,value
495,osmo1ew02nl40jqspvfe23xmdc4xmqvnelz3929nrrx,31235.395152
733,osmo1lvxcql4c9yeec8spuaspeq5j5slnd27g6zdh0a,22659.414081
791,osmo1nl09n46sfnk0z8jk33mvu9u6lz5ymqj4hmj84x,13756.853823
647,osmo1jh0urskmdrrhfgm53d0ddnd506stu63w95d3l6,11609.317352
582,osmo1gw6xkw2pvxtt3ea83gs7gdhv7zvhzmk9yyl3xk,8562.143041
...,...,...
249,osmo178swfpjp3qwes3wj8ffzqla9m56ewzxm0k49xk,0.000998
762,osmo1mvccc0xt6g8lm5stldfzmjfdg4vhq9rd90df7f,0.000690
883,osmo1r0gtpf5s4v3ryujwza96qe2v606cgywfxrsjya,0.000383
768,osmo1mxpjsl4gppu3al5c0hm5ttm8hf9zpdhctafmxy,0.000297


In [17]:
agg_join_tokens = (cleaned_join_df.groupby(['symbol'])[['amount', 'value']]
                                  .sum()
                                  .reset_index()
                                  .sort_values(by=['value'], ascending=False))
agg_join_tokens.to_csv("data/agg_join_tokens.csv", index=False)
agg_join_tokens

,symbol,amount,value
27,OSMO,7.113284e+04,79534.202420
43,axlWBTC,5.045495e-01,15649.481953
24,MNTL,1.662249e+05,15629.871855
44,axlWETH,8.343544e+00,15206.016875
42,axlUSDC,1.050697e+04,10506.974169
17,ION,1.034521e+01,9323.538023
1,ATOM,9.438959e+02,8891.369179
40,XPRT,4.726250e+03,6529.882249
41,axlDAI,4.042419e+03,4022.289613
12,EVMOS,1.977356e+03,3596.908690


In [18]:
agg_join_pools = (cleaned_join_df.groupby(['pool_id', 'symbol'])[['amount', 'value']]
                                 .sum()
                                 .reset_index()
                                 .sort_values(by=['value'], ascending=False))
agg_join_pools.to_csv("data/agg_join_pools.csv", index=False)
agg_join_pools

,pool_id,symbol,amount,value
110,690,OSMO,14227.435914,15907.811764
109,690,MNTL,166224.894603,15629.871855
115,712,axlWBTC,0.503852,15627.863283
113,704,axlWETH,8.331704,15184.439856
114,712,OSMO,13520.254377,15117.106339
...,...,...,...,...
86,637,OSMO,0.012185,0.013624
77,627,OSMO,0.004885,0.005462
78,627,SOMM,0.011336,0.005456
42,580,JUNO,0.000001,0.000004


## Assets withdrawn from Exploit

In [19]:
potential_exploit_value = valid_transactions.loc[np.isin(valid_transactions.sender, potentially_malicious_users)]
first_token = potential_exploit_value.drop(columns=['token_2_denom', 'token_2_amount']).rename(columns={'token_1_denom':'denom', 'token_1_amount': 'amount'})
second_token = potential_exploit_value.drop(columns=['token_1_denom', 'token_1_amount']).rename(columns={'token_2_denom':'denom', 'token_2_amount': 'amount'})

flat_txs = pd.concat([first_token, second_token])
flat_txs.amount = flat_txs.amount.astype(float)
flat_txs.shares = flat_txs.shares.astype(float, copy=False)

In [20]:
named_txs = token_map_df.merge(flat_txs)
named_txs['amount'] = named_txs.amount/(10**named_txs.exponent)
named_txs = named_txs.merge(halt_price_df)
named_txs['value'] = named_txs.price * named_txs.amount
cleaned_df = named_txs.drop(columns=['denom', 'exponent'])

In [21]:
net_diff = cleaned_df.copy()
net_diff['amount'] = np.where(net_diff.type == 'Join', -1*net_diff.amount, net_diff.amount)
net_diff['value'] = np.where(net_diff.type == 'Join', -1*net_diff.value, net_diff.value)

In [22]:
agg_net_diff = (net_diff[['value']].sum())
agg_net_diff

value    5.038084e+06
dtype: float64

In [23]:
agg_net_diff_sender_tokens = (net_diff.groupby(['sender', 'symbol'])[['amount', 'value']]
                                      .sum()
                                      .reset_index()
                                      .sort_values(by=['value'], ascending=False))
agg_net_diff_sender_tokens.to_csv("data/agg_net_diff_sender_tokens.csv", index=False)
agg_net_diff_sender_tokens

,sender,symbol,amount,value
10,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,OSMO,1.332914e+06,1.490342e+06
9,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,ATOM,1.365926e+05,1.286683e+06
33,osmo1hq8tlgq0kqz9e56532zghdhz7g8gtjymdltqer,axlUSDC,7.851486e+05,7.851486e+05
32,osmo1hq8tlgq0kqz9e56532zghdhz7g8gtjymdltqer,OSMO,6.892692e+05,7.706775e+05
11,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,axlWBTC,4.216830e+00,1.307923e+05
...,...,...,...,...
6,osmo16h0yyryfjmmhgj3ct6ndl7gcy4r3krf0gfrzky,ATOM,-2.749048e+00,-2.589565e+01
1,osmo10rgny3xd5fvg02wnmy4h9zuktnfthvrlwdjwf5,OSMO,-3.439947e+01,-3.846233e+01
0,osmo10rgny3xd5fvg02wnmy4h9zuktnfthvrlwdjwf5,ATOM,-4.354271e+00,-4.101663e+01
41,osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte,LUNA,-2.688360e+07,-1.306655e+03


In [24]:
agg_net_diff_senders = (net_diff.groupby(['sender'])[['value']]
                                .sum()
                                .reset_index()
                                .sort_values(by=['value'], ascending=False))
agg_net_diff_senders.to_csv("data/agg_net_diff_senders.csv", index=False)
agg_net_diff_senders

,sender,value
4,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,3.034353e+06
10,osmo1hq8tlgq0kqz9e56532zghdhz7g8gtjymdltqer,1.555826e+06
18,osmo1v44mqmhvtn8cw373xv0hw6npddccnr70lqsk9s,1.631672e+05
1,osmo10t26acjmemggsahq6uvyucm4tj3z0mhz23ljh2,1.190660e+05
20,osmo1yglld3aary7lnrrn2dz7la84kmnmen4kpsxzay,7.954177e+04
11,osmo1jdh7eeyaar0tyask0r6w2228uh5wrd0pxtcfwr,6.238636e+04
17,osmo1ux20lcw7et2j8kl8gfdep78lacew4dqqjp5dvp,2.464548e+04
15,osmo1tg70tuzekpd376dpqr68yx5a7r709w6x8jtxha,9.932608e+02
14,osmo1pfpk6rsgc0x7g8q7y0yre86ej2enr8rhv3p8dq,7.860224e+02
12,osmo1jfa3vw3r50vyww2qwklxqn6qu06d3uf7vc63qm,8.649217e+01


In [25]:
agg_net_diff_tokens = (net_diff.groupby(['symbol'])[['amount', 'value']]
                               .sum()
                               .reset_index()
                               .sort_values(by=['value'], ascending=False))
agg_net_diff_tokens.to_csv("data/agg_net_diff_tokens.csv", index=False)
agg_net_diff_tokens

,symbol,amount,value
11,OSMO,2.220503e+06,2.482763e+06
0,ATOM,1.366605e+05,1.287323e+06
15,axlUSDC,8.983485e+05,8.983485e+05
16,axlWBTC,7.867308e+00,2.440183e+05
17,axlWETH,6.941911e+01,1.265156e+05
5,EVMOS,2.158885e+02,3.927118e+02
7,ION,2.883600e-02,2.598820e+01
2,BTSG,1.571676e+02,2.621648e+00
8,JUNO,5.483470e-01,2.047749e+00
12,SCRT,2.036900e-01,2.655628e-01


In [26]:
agg_net_diff_pools = (net_diff.groupby(['pool_id', 'symbol'])[['amount', 'value']]
                              .sum()
                              .reset_index()
                              .sort_values(by=['value'], ascending=False))
agg_net_diff_pools.to_csv("data/agg_net_diff_pools.csv", index=False)
agg_net_diff_pools

,pool_id,symbol,amount,value
0,1,ATOM,1.366606e+05,1.287324e+06
1,1,OSMO,1.112731e+06,1.244154e+06
30,678,axlUSDC,8.983485e+05,8.983485e+05
29,678,OSMO,7.894525e+05,8.826933e+05
38,712,axlWBTC,7.867308e+00,2.440183e+05
37,712,OSMO,2.106481e+05,2.355273e+05
36,704,axlWETH,6.941911e+01,1.265156e+05
35,704,OSMO,1.084945e+05,1.213086e+05
39,722,EVMOS,2.158885e+02,3.927118e+02
40,722,OSMO,3.504178e+02,3.918050e+02
